# Sorted Spikes Decoding

The mechanics of decoding with sorted spikes are largely similar to those of decoding with unsorted spikes. You should familiarize yourself with the [clusterless decoding tutorial](./42_Decoding_Clusterless.ipynb) before proceeding with this one.

The elements we will need to decode with sorted spikes are:
- `PositionGroup`
- `SortedSpikesGroup`
- `DecodingParameters`
- `encoding_interval`
- `decoding_interval`

This time, instead of extracting waveform features, we can proceed directly from the SpikeSortingOutput table to specify which units we want to decode. The rest of the decoding process is the same as before.


## SortedSpikesGroup

In [1]:
from pathlib import Path
import datajoint as dj

dj.config.load(
    Path("../dj_local_conf.json").absolute()
)  # load config for database connection info

In [2]:
from spyglass.spikesorting.merge import SpikeSortingOutput
import spyglass.spikesorting.v1 as sgs


nwb_copy_file_name = "mediumnwb20230802_.nwb"

sorter_keys = {
    "nwb_file_name": nwb_copy_file_name,
    "sorter": "clusterless_thresholder",
    "sorter_param_name": "default_clusterless",
}

(sgs.SpikeSortingSelection & sorter_keys) * SpikeSortingOutput.CurationV1

[2024-01-02 18:41:02,417][INFO]: Connecting root@localhost:3306
[2024-01-02 18:41:02,501][INFO]: Connected root@localhost:3306


sorting_id,merge_id,recording_id,sorter,sorter_param_name,nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,curation_id
0cbd8579-6c48-4506-a116-e27e9b89f174,86acdb0f-84f0-73a2-a851-1f8305cd2e41,458ba3c2-3a08-4291-af10-c74d823330d4,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,b52d303b-12b1-4584-8f35-63dde543836c,0
22283413-433c-4c6f-b2fa-f82a10327df7,46829e10-1984-99a1-65a3-2b485a2f037f,8e3daa47-2ee6-435f-892b-f095b1c5aa1a,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,1536b082-018d-4674-b562-cac09d298b7f,0
22e6bc74-e755-440c-a507-f9292fd494c9,ec308784-2bfb-dd90-147c-e4d44e5f649b,7fb98af6-d486-439f-ae1b-7abdfddae56b,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,b1a34880-c87b-403f-8a3b-c346e614c782,0
32fa3502-7fa9-469b-a7a1-0e0e670fe28e,4b3065e5-76c2-bd48-32a1-ae62484f9314,ef989f5a-3cf4-488d-be1f-660970fdfd69,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,4fa14f8e-14a2-49ce-a1b4-6c447fdc3a1e,0
338442ef-821c-401e-91ba-8eec27490701,609aeb54-dc2e-52d3-91bf-1728e0a2cf09,86d39675-d6b0-4697-b336-9b2b1766d8f3,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,3824e250-27e5-4cc5-a49d-56d9e37b3ad8,0
43495249-ab6b-4067-b04a-11401b998215,88492b1c-f4a9-9669-bb5b-7f1573015187,ded4b85c-a2f8-465d-ab21-504905c06403,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,7f38783a-215f-47c1-853b-2e1ddc941d7f,0
43a6942c-668e-44a1-aa5b-a7aebc5c424a,f515c07f-fc80-b28a-750d-d0d5491259f4,078776e3-1b9c-4755-bef8-b9201bcdd717,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,ac7875e6-a370-4cf3-a74e-263f0d98a17a,0
4986cd16-515f-441a-8653-36cf3a312ca0,f4e29a80-ec96-dbe8-7081-425ac311b74c,db9d73cf-f9e2-46b4-8eb7-a8d059d99bf6,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,d630e3bb-10b2-4466-9c20-1db14565bcf4,0
59e06873-aae3-438a-8bc1-2988315b3d7e,d7754d5f-af01-19f4-3fdc-c9635081667a,aeda79a6-8442-4a39-93b7-bce6da6fcacd,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,6bea1980-8ea0-4160-afc3-aef93743fb9d,0
67b0fafd-693f-4a26-a20b-100c0a4731a7,2567bf67-bc67-47a5-aa2a-2bce19da232d,47337655-182c-4c9d-b79d-ea0c6ce51b34,clusterless_thresholder,default_clusterless,mediumnwb20230802_.nwb,b7fc2304-9cbf-4d85-8028-39cab674273a,0


In [3]:
spikesorting_merge_ids = (
    (sgs.SpikeSortingSelection & sorter_keys) * SpikeSortingOutput.CurationV1
).fetch("merge_id")

spikesorting_merge_ids

array([UUID('86acdb0f-84f0-73a2-a851-1f8305cd2e41'),
       UUID('46829e10-1984-99a1-65a3-2b485a2f037f'),
       UUID('ec308784-2bfb-dd90-147c-e4d44e5f649b'),
       UUID('4b3065e5-76c2-bd48-32a1-ae62484f9314'),
       UUID('609aeb54-dc2e-52d3-91bf-1728e0a2cf09'),
       UUID('88492b1c-f4a9-9669-bb5b-7f1573015187'),
       UUID('f515c07f-fc80-b28a-750d-d0d5491259f4'),
       UUID('f4e29a80-ec96-dbe8-7081-425ac311b74c'),
       UUID('d7754d5f-af01-19f4-3fdc-c9635081667a'),
       UUID('2567bf67-bc67-47a5-aa2a-2bce19da232d'),
       UUID('d65a1bf3-797d-b01f-e8be-2cea90b14c20'),
       UUID('92c336ee-81f4-0af9-4f60-9bc32e71bc9f'),
       UUID('aa8bc575-0715-69e9-5da7-313a0e1ee769'),
       UUID('26310ce7-9ac3-4159-99f8-a3ad17037235'),
       UUID('7355bdf3-f31c-4c22-1a09-50d9f6f5f037'),
       UUID('189fb8c6-f964-00a9-f392-a9dbb138ea63'),
       UUID('c4f24219-c023-8783-df53-2bbc88c9ad9c'),
       UUID('411dff13-44f0-3e03-e867-689ae275e418'),
       UUID('153954b2-b230-cb1f-749d-f977a22ea

In [4]:
from spyglass.decoding.v1.sorted_spikes import SortedSpikesGroup

SortedSpikesGroup()

nwb_file_name name of the NWB file,sorted_spikes_group_name


In [5]:
SortedSpikesGroup().create_group(
    group_name="test_group",
    nwb_file_name=nwb_copy_file_name,
    keys=[
        {"spikesorting_merge_id": merge_id}
        for merge_id in spikesorting_merge_ids
    ],
)

SortedSpikesGroup & {
    "nwb_file_name": nwb_copy_file_name,
    "sorted_spikes_group_name": "test_group",
}

nwb_file_name name of the NWB file,sorted_spikes_group_name
mediumnwb20230802_.nwb,test_group


In [6]:
SortedSpikesGroup.SortGroup & {
    "nwb_file_name": nwb_copy_file_name,
    "sorted_spikes_group_name": "test_group",
}

nwb_file_name name of the NWB file,sorted_spikes_group_name,spikesorting_merge_id
mediumnwb20230802_.nwb,test_group,00763b68-d663-c446-0555-1f2622d7da50
mediumnwb20230802_.nwb,test_group,03954edd-f8fd-3dd9-cd10-f0eee47d6b3d
mediumnwb20230802_.nwb,test_group,0720e5f2-625e-09d2-b522-ca2652c09f2a
mediumnwb20230802_.nwb,test_group,153954b2-b230-cb1f-749d-f977a22eaae9
mediumnwb20230802_.nwb,test_group,189fb8c6-f964-00a9-f392-a9dbb138ea63
mediumnwb20230802_.nwb,test_group,2567bf67-bc67-47a5-aa2a-2bce19da232d
mediumnwb20230802_.nwb,test_group,26310ce7-9ac3-4159-99f8-a3ad17037235
mediumnwb20230802_.nwb,test_group,411dff13-44f0-3e03-e867-689ae275e418
mediumnwb20230802_.nwb,test_group,43a98eab-1fa6-184b-1f09-2e923984b03a
mediumnwb20230802_.nwb,test_group,46829e10-1984-99a1-65a3-2b485a2f037f


## Model parameters

As before we can specify the model parameters. The only difference is that we will use the `ContFragSortedSpikesClassifier` instead of the `ContFragClusterlessClassifier`.

In [7]:
from spyglass.decoding.v1.core import DecodingParameters
from non_local_detector.models import ContFragSortedSpikesClassifier


DecodingParameters.insert1(
    {
        "decoding_param_name": "contfrag_sorted",
        "decoding_params": ContFragSortedSpikesClassifier(),
        "decoding_kwargs": dict(),
    },
    skip_duplicates=True,
)

DecodingParameters()

decoding_param_name a name for this set of parameters,decoding_params initialization parameters for model,decoding_kwargs additional keyword arguments
contfrag_clusterless,=BLOB=,=BLOB=
contfrag_sorted,=BLOB=,=BLOB=


### 1D Decoding

As in the clusterless notebook, we can decode 1D position if we specify the `track_graph`, `edge_order`, and `edge_spacing` parameters in the `Environment` class constructor. See the [clusterless decoding tutorial](./42_Decoding_Clusterless.ipynb) for more details.

## Decoding

Now we can decode the position using the sorted spikes using the `SortedSpikesDecodingSelection` table. Here we assume that `PositionGroup` has been specified as in the clusterless decoding tutorial.

In [8]:
from spyglass.decoding.v1.sorted_spikes import SortedSpikesDecodingSelection

SortedSpikesDecodingSelection()

nwb_file_name name of the NWB file,sorted_spikes_group_name,position_group_name,decoding_param_name a name for this set of parameters,encoding_interval descriptive name of this interval list,decoding_interval descriptive name of this interval list,estimate_decoding_params whether to estimate the decoding parameters


In [9]:
selection_key = {
    "sorted_spikes_group_name": "test_group",
    "position_group_name": "test_group",
    "decoding_param_name": "contfrag_sorted",
    "nwb_file_name": "mediumnwb20230802_.nwb",
    "encoding_interval": "pos 0 valid times",
    "decoding_interval": "test decoding interval",
    "estimate_decoding_params": False,
}

SortedSpikesDecodingSelection.insert1(
    selection_key,
    skip_duplicates=True,
)

In [10]:
from spyglass.decoding.v1.sorted_spikes import SortedSpikesDecodingV1

SortedSpikesDecodingV1.populate(selection_key)

[2024-01-02 18:41:06,204][WARNING]: Skipped checksum for file with hash: 77a1423b-2a35-00bf-14d5-d21a8a634688, and path: /Users/edeno/Documents/GitHub/spyglass/DATA/analysis/mediumnwb20230802/mediumnwb20230802_5C8F8R2R3G.nwb
[2024-01-02 18:41:06,447][WARNING]: Skipped checksum for file with hash: b12a9d1d-d019-7c0e-09bf-355936d14915, and path: /Users/edeno/Documents/GitHub/spyglass/DATA/analysis/mediumnwb20230802/mediumnwb20230802_DRQ7MITSST.nwb
[2024-01-02 18:41:06,657][WARNING]: Skipped checksum for file with hash: c8f4786b-9ef7-61f7-cae0-251e84c59317, and path: /Users/edeno/Documents/GitHub/spyglass/DATA/analysis/mediumnwb20230802/mediumnwb20230802_XXD817HX5I.nwb
[2024-01-02 18:41:06,855][WARNING]: Skipped checksum for file with hash: 6907761c-fb37-6528-56d7-507d5525a69b, and path: /Users/edeno/Documents/GitHub/spyglass/DATA/analysis/mediumnwb20230802/mediumnwb20230802_JN3OG7ZA5E.nwb
[2024-01-02 18:41:07,048][WARNING]: Skipped checksum for file with hash: 1767224a-ebf4-819e-deb3-67c

Encoding models:   0%|          | 0/23 [00:00<?, ?cell/s]

Non-Local Likelihood:   0%|          | 0/23 [00:00<?, ?cell/s]

/Users/edeno/miniconda3/envs/spyglass/lib/python3.9/site-packages/non_local_detector/models/base.py:780: FutureWarning: the `pandas.MultiIndex` object(s) passed as 'state_bins' coordinate(s) or data variable(s) will no longer be implicitly promoted and wrapped into multiple indexed coordinates in the future (i.e., one coordinate for each multi-index level + one dimension coordinate). If you want to keep this behavior, you need to first wrap it explicitly using `mindex_coords = xarray.Coordinates.from_pandas_multiindex(mindex_obj, 'dim')` and pass it as coordinates, e.g., `xarray.Dataset(coords=mindex_coords)`, `dataset.assign_coords(mindex_coords)` or `dataarray.assign_coords(mindex_coords)`.
  results = xr.Dataset(
/Users/edeno/miniconda3/envs/spyglass/lib/python3.9/site-packages/xarray/namedarray/core.py:487: UserWarning: Duplicate dimension names present: dimensions {'states'} appear more than once in dims=('states', 'states'). We do not yet support duplicate dimension names, but we

We verify that the results have been inserted into the `DecodingOutput` merge table.

In [11]:
from spyglass.decoding.decoding_merge import DecodingOutput

DecodingOutput.SortedSpikesDecodingV1 & selection_key

merge_id,nwb_file_name name of the NWB file,sorted_spikes_group_name,position_group_name,decoding_param_name a name for this set of parameters,encoding_interval descriptive name of this interval list,decoding_interval descriptive name of this interval list,estimate_decoding_params whether to estimate the decoding parameters
650316f6-e19b-6195-5aea-0f46bc503fa3,mediumnwb20230802_.nwb,test_group,test_group,contfrag_sorted,pos 0 valid times,test decoding interval,0


We can load the results as before:

In [12]:
results = (SortedSpikesDecodingV1 & selection_key).load_results()
results

[2024-01-02 18:42:08,689][WARNING]: Skipped checksum for file with hash: 3a5b5656-70f4-d216-4e33-116b976598a4, and path: /Users/edeno/Documents/GitHub/spyglass/DATA/analysis/mediumnwb20230802/mediumnwb20230802_a65e09a0-907b-4e19-8d79-2262edad3495.nc
/Users/edeno/miniconda3/envs/spyglass/lib/python3.9/site-packages/xarray/namedarray/core.py:487: UserWarning: Duplicate dimension names present: dimensions {'states'} appear more than once in dims=('states', 'states'). We do not yet support duplicate dimension names, but we do allow initial construction of the object. We recommend you rename the dims immediately to become distinct, as most xarray functionality is likely to fail silently if you do not. To rename the dimensions you will need to set the ``.dims`` attribute of each variable, ``e.g. var.dims=('x0', 'x1')``.
  warnings.warn(
/Users/edeno/miniconda3/envs/spyglass/lib/python3.9/site-packages/xarray/namedarray/core.py:487: UserWarning: Duplicate dimension names present: dimensions {

<xarray.Dataset>
Dimensions:                      (state_ind: 26668, dim_0: 26668, time: 451,
                                  states: 2, intervals: 1, state_bins: 26668)
Coordinates:
  * state_ind                    (state_ind) int32 0 0 0 0 0 0 0 ... 1 1 1 1 1 1
  * time                         (time) float64 1.626e+09 ... 1.626e+09
  * states                       (states) object 'Continuous' 'Fragmented'
    environments                 (states) object ...
    encoding_groups              (states) int32 ...
  * state_bins                   (state_bins) object MultiIndex
  * state                        (state_bins) object 'Continuous' ... 'Fragme...
  * x_position                   (state_bins) float64 29.02 29.02 ... 262.7
  * y_position                   (state_bins) float64 0.4493 2.445 ... 224.0
Dimensions without coordinates: dim_0, intervals
Data variables:
    initial_conditions           (dim_0) float64 ...
    discrete_state_transitions   (states, states) float64 ...
    acausal_posterior            (intervals, time, state_bins) float32 ...
    acausal_state_probabilities  (intervals, time, states) float64 ...
Attributes:
    marginal_log_likelihoods:  -16366.832